In [1]:
import pymysql
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
connection = pymysql.connect(host='localhost',user='root',password='******',database='pharma')
connection

In [3]:
query = 'select * from bill'
df = pd.read_sql(query,connection)
df.head(2)

,bill_id,drug_id,customer_id,store_id,quantity,bill_date
0,70482955,89036292,74787717,717,3,2018-12-05
1,70482955,20390694,74787717,717,5,2018-12-05


In [14]:
# Q1: What is the total quantity of drugs sold in a particular store location?

query1 = 'SELECT ' \
         's.store_location, COUNT(d.drug_id) AS total_drug ' \
         'FROM ' \
         'drug d ' \
         'JOIN bill b ON d.drug_id = b.drug_id ' \
         'JOIN store s ON s.store_id = b.store_id ' \
         'GROUP BY s.store_location'

ans1 = pd.read_sql(query1,connection)
ans1

,store_location,total_drug
0,Texas,3365
1,Ohio,6666
2,Florida,4922
3,Washington,1789
4,New York,6594
5,Tennessee,5146
6,California,3440
7,Virginia,3576
8,Hawaii,1619
9,North Carolina,1707


In [17]:
# Q2: Which customer has purchased the most number of drugs, and what is the total amount they have spent?

query2 = 'select c.first_name, c.last_name, count(d.drug_id) as total_drug, sum(d.price) as total_amount '\
        'from customer c join bill b '\
        'on  c.customer_id = b.customer_id '\
        'join drug d on d.drug_id = b.drug_id '\
        'group by c.first_name, c.last_name '\
        'order by total_amount desc '\
        'limit 10'
ans2 = pd.read_sql(query2,connection)
ans2

,first_name,last_name,total_drug,total_amount
0,Englebert,Agar,97,527597.0
1,Gavrielle,Dillinger,70,392799.0
2,Matilda,Ahren,70,380440.0
3,Garek,Shoreman,65,348511.0
4,Dirk,Stivens,60,338891.0
5,Kristoforo,Eisenberg,65,331148.0
6,Marlow,Lyle,55,315749.0
7,Timmi,Scrafton,54,315448.0
8,Dari,Seward,51,311548.0
9,Benni,Rzehor,55,309924.0


In [19]:
# Q3: What is the total revenue generated by a specific store location?

query3 = 'select s.store_location, sum(b.quantity * d.price) as total_revenue from drug d join bill b '\
        'on d.drug_id = b.drug_id '\
        'join store s on s.store_id = b.store_id '\
        'group by s.store_location '\
        'order by total_revenue desc '\
        'limit 5'
ans3 = pd.read_sql(query3,connection)
ans3

,store_location,total_revenue
0,New York,105870492.0
1,Ohio,105056343.0
2,Tennessee,81122888.0
3,Florida,76273379.0
4,Virginia,54981635.0


In [20]:
# Q4: Which drug has been sold the most across all stores?

query4 = 'select d.drug_name, count(c.customer_id) as total_drug_sales from drug d join bill b '\
        'on d.drug_id = b.drug_id '\
        'join customer c on c.customer_id = b.customer_id '\
        'group by d.drug_name '\
        'order by total_drug_sales desc '\
        'limit 10 '
ans4 = pd.read_sql(query4,connection)
ans4

,drug_name,total_drug_sales
0,BENZALKONIUM CHLORIDE,1007
1,IBUPROFEN,846
2,NAPROXEN SODIUM,630
3,ACETAMINOPHEN,614
4,TITANIUM DIOXIDE,500
5,SODIUM FLUORIDE,483
6,HYDROCODONE BITARTRATE AND ACETAMINOPHEN,470
7,ALCOHOL,458
8,PYRITHIONE ZINC,453
9,GABAPENTIN,345


In [21]:
# Q5: What are the details (name, price, and manufacturer) of the most expensive drug sold in a particular store?

query5 = 'select drug_name, manufacturer, price from drug '\
        'order by price desc '\
        'limit 10;'
ans5 = pd.read_sql(query5,connection)
ans5

,drug_name,manufacturer,price
0,MECLIZINE HCL,WALGREEN CO.,9988
1,DIMENHYDRINATE,MEDTECH PRODUCTS INC.,9977
2,"AVOBENZONE, OCTISALATE, ZINC OXIDE, OCTINOXATE...",VENTURA CORPORATION LTD,9959
3,"AVOBENZONE, OCTINOXATE, OCTISALATE, AND OCTOCR...",GUTHY-RENKER LLC,9929
4,CEFIXIME,LUPIN PHARMA,9907
5,ATORVASTATIN CALCIUM,CARILION MATERIALS MANAGEMENT,9899
6,ZALEPLON,BRYANT RANCH PREPACK,9895
7,CLOBETASOL PROPIONATE,REBEL DISTRIBUTORS CORP,9891
8,LEVETIRACETAM,"PD-RX PHARMACEUTICALS, INC.",9860
9,ALLSCALE,"ANTIGEN LABORATORIES, INC.",9858


In [4]:
# Q6: How many customers have made purchases in each store, and which store has the highest number of customers?

query6 = 'select '\
    's.store_id, count(c.customer_id) as number_of_customer  from store s join bill b '\
    'on s.store_id = b.store_id '\
    'join customer c on c.customer_id = b.customer_id '\
    'group by store_id '\
    'order by number_of_customer desc '\
    'limit 5'

ans6 = pd.read_sql(query6,connection)
ans6

,store_id,number_of_customer
0,486,1831
1,504,1812
2,951,1790
3,875,1789
4,379,1774


In [12]:
# Q7): What is the average quantity of drugs sold per transaction at a specific store?

query7 = '''SELECT 
    store_id,
    AVG(total_quantity) AS avg_quantity_per_transaction
FROM
    (SELECT 
        s.store_id, c.customer_id, SUM(b.quantity) AS total_quantity
    FROM
        store s
    JOIN bill b ON s.store_id = b.store_id
    JOIN customer c ON b.customer_id = c.customer_id
    GROUP BY s.store_id , c.customer_id) AS transaction_totals
GROUP BY store_id
limit 5
'''

ans7 = pd.read_sql(query7,connection)
ans7

,store_id,avg_quantity_per_transaction
0,080,16.7196
1,087,17.4237
2,140,16.5263
3,147,17.3860
4,161,17.7677


In [13]:
# Q8: Which drug was purchased the most by customers under the age of 30?
query8 = '''select d.manufacturer, d.drug_name, d.price, s.store_location from drug d join bill b
on b.drug_id = d.drug_id
join store s on s.store_id = b.store_id
limit 5
'''

ans8 = pd.read_sql(query8,connection)
ans8

,manufacturer,drug_name,price,store_location
0,"AVION PHARMACEUTICALS, LLC","ASCORBIC ACID, CHOLECALCIFEROL, .ALPHA.-TOCOPH...",7584,Texas
1,P AND L DEVELOPMENT OF NEW YORK COMPANY,DIPHENHYDRAMINE HYDROCHLORIDE,8995,Texas
2,ENERGETIX CORP,HOMEOPATHIC GEL,4017,Texas
3,"DISPENSING SOLUTIONS, INC.",OLANZAPINE,8034,Texas
4,ELI LILLY AND COMPANY,PEMETREXED DISODIUM,3183,Texas


In [14]:
# Q9: What is the most common drug purchased by male customers in a given store?

query9 = '''select drug_name, count(drug_name) as most_purchased_drug_by_male from  drug d join bill b 
on d.drug_id = b.drug_id 
join customer c on c.customer_id = b.customer_id
where gender = 'M'
group by drug_name
order by most_purchased_drug_by_male desc
limit 5'''

ans9 = pd.read_sql(query9,connection)
ans9

,drug_name,most_purchased_drug_by_male
0,BENZALKONIUM CHLORIDE,476
1,IBUPROFEN,423
2,NAPROXEN SODIUM,299
3,ACETAMINOPHEN,281
4,TITANIUM DIOXIDE,234
